比賽簡介：

  概述：
    歡迎來到2912年，在這裡需要你的數據科學技能來解決一個宇宙之謎。我們收到了來自四光年外的傳輸消息，情況看起來並不好。

    鐵達尼號宇宙飛船是一個月前發射的星際客輪。船上有近13,000名乘客，這艘船開始了處女航，將移民從我們的太陽系運送到三顆圍繞附近恆星運行的新的可居住的系外行星。

    鐵達尼號宇宙飛船在繞過半人馬座阿爾法星前往第一個目的地——55 Cancri E的途中，闖進了隱藏在塵埃雲中時空異常區域。可悲的是，它遇到了與1000年前同名遊輪相同的命運。雖然飛船完好無損，但幾乎一半的乘客被運送到另一個維度！

  資料來源：
    https://www.kaggle.com/competitions/spaceship-titanic/overview
    https://www.cnblogs.com/bokeyuan199/p/16997206.html


為什麼選擇這個比賽：與機器學習高度相關，且相較其他競賽較為基礎，與剛踏入機器學習領域的我較為契合。


資料集、目標介紹：

  資料集：
    此比賽所使用的數據集中有test.csv / train.csv兩個文件，結果以sample_submission.csv為範本參考。

  目標介紹：
    為了幫助救援人員找回失踪的乘客，您面臨的挑戰是使用從宇宙飛船損壞的計算機系統中恢復的記錄來預測哪些乘客被傳送到時空異常區域。拯救失蹤人員改變歷史！

實作過程如下：程式碼與結果


跟上課內容的關聯性：處理缺失值(0、中位數填補)、訓練過程相似、數據可視化(圖表轉換)


延伸學習了哪些：利用其他圖表呈現以及訓練模型作為預測結果

比賽結果說明：
  分數：0.74748 / 排名：1896 (共2429)


推測還可能從哪些方式改善：使用不同的訓練模型


不同的嘗試與結果分析：
  若將以下連結的程式碼上傳至Kaggle，可能會得到更高的準確率，因為他填入缺失值的方式是使用姓名當中的稱謂中位數來填補，比起填中位數要準確的多。(但因著作權問題沒有實際測試，以上為參考蒐集到的相關資料)
    程式碼連結：https://github.com/YLTsai0609/Kaggle-Titanic-Top3-percent/blob/master/Titanic%20Top%203%25%20on%20Medium.ipynb
    相關資料來源：https://yulongtsai.medium.com/https-medium-com-yulongtsai-titanic-top3-8e64741cc11f

    

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns